## Import Libraries

In [1]:
from UFCStats.queries import DatabaseQuery

import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import dataframe_image as dfi

import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

import pickle

import warnings
warnings.filterwarnings('ignore')

from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_samples, silhouette_score

from scipy.cluster.hierarchy import dendrogram
from scipy.spatial import distance

from math import ceil

from bs4 import BeautifulSoup
import requests
import time, os

## Run Database Queries

In [2]:
# Run queries to add db data to pandas dataframes
DQ = DatabaseQuery()

def query_to_df(query):
    df = pd.DataFrame(query.fetchall())
    df.columns = query.keys()
    return df

In [3]:
# Query database for fight details
fight_cols = ['fight_id', 'event_id', 'weightclass', 'rds_sched', 'rd_ended', 'method', 'bonus', 
              'fighter_1_id', 'fighter_2_id', 'winner_id', 'tapology_rank', 'deductions']

fight_col_str = ', '.join(fight_cols)

fight_details = query_to_df(DQ.engine.execute('SELECT ' + fight_col_str + ' FROM fights'))

# Query database for round information
round_details = query_to_df(DQ.engine.execute('SELECT rd_id, fight_id, rd_num, rd_length FROM rounds'))

# Query database for event dates
event_dates = query_to_df(DQ.engine.execute('SELECT event_id, event_date FROM events'))

# Query database for fighter information
fighter_details = query_to_df(DQ.engine.execute('SELECT * FROM fighters'))

# Query database for fight statistics by round
result_cols = ['rd_id', 'fighter_id', 'kd', 'sig_head_land', 'sig_head_att', 'sig_body_land', 
               'sig_body_att', 'sig_leg_land', 'sig_leg_att', 'sig_dist_land', 'sig_dist_att',
               'sig_clinch_land', 'sig_clinch_att', 'sig_ground_land', 'sig_ground_att', 
               'total_strike_land', 'total_strike_att', 'takedown_land', 'takedown_att', 'sub_att',
               'reversals', 'ctrl_time', 'kd_taken', 'sig_head_taken', 'sig_head_seen', 
               'sig_body_taken', 'sig_body_seen', 'sig_leg_taken', 'sig_leg_seen', 'sig_dist_taken',
               'sig_dist_seen', 'sig_clinch_taken', 'sig_clinch_seen', 'sig_ground_taken', 
               'sig_ground_seen', 'total_strike_taken', 'total_strike_seen', 'takedown_taken', 
               'takedown_seen', 'sub_seen', 'reversals_taken', 'ctrl_time_taken']

result_col_str = ', '.join(result_cols)

stats_by_round = query_to_df(DQ.engine.execute('SELECT ' + result_col_str + ' FROM round_results'))

## Clean Up Stats By Round

In [4]:
# Add columns for significant strikes landed and attempted by the fighter and the opponent
stats_by_round['total_sig_land'] = stats_by_round['sig_head_land'] + stats_by_round['sig_body_land'] + stats_by_round['sig_leg_land']
stats_by_round['total_sig_att'] = stats_by_round['sig_head_att'] + stats_by_round['sig_body_att'] + stats_by_round['sig_leg_att']
stats_by_round['total_sig_land_opp'] = stats_by_round['sig_head_taken'] + stats_by_round['sig_body_taken'] + stats_by_round['sig_leg_taken']
stats_by_round['total_sig_att_opp'] = stats_by_round['sig_head_seen'] + stats_by_round['sig_body_seen'] + stats_by_round['sig_leg_seen']

In [5]:
# Merge round number and round length to fight stats table
stats_by_round = stats_by_round.merge(round_details[['rd_id', 'rd_num', 'rd_length']],
                                      on='rd_id',
                                      how='left'
                                     )

In [6]:
# Drop all rounds where control time was not provided on UFCStats.com (NaN)
stats_by_round.dropna(subset=['ctrl_time', 'ctrl_time_taken'], inplace=True)

In [7]:
# Create columns for neutral time and control time plus neutral time
stats_by_round['ctrl_plus_neutral'] = stats_by_round['rd_length'] - stats_by_round['ctrl_time_taken']
stats_by_round['neutral_time'] = stats_by_round['ctrl_plus_neutral'] - stats_by_round['ctrl_time']

## Determine Recency of Fights

In [8]:
# Create fight recency table, showing an integer representing how fights into the past (most recent fight = 1)
f1_fights = fight_details[['fight_id', 'event_id', 'fighter_1_id']].rename(columns={'fighter_1_id' : 'fighter_id'})
f2_fights = fight_details[['fight_id', 'event_id', 'fighter_2_id']].rename(columns={'fighter_2_id' : 'fighter_id'})

fight_recency = pd.concat([f1_fights, f2_fights]).reset_index(drop=True)
fight_recency['fight_fighter'] = fight_recency['fight_id'] + '-' + fight_recency['fighter_id']
fight_recency = fight_recency.merge(event_dates, on='event_id').sort_values(['fighter_id', 'event_date'], ascending=False)

fight_recency['recency'] = fight_recency.groupby('fighter_id').cumcount() + 1

In [9]:
# Create fight_id column and merge rounds scheduled
stats_by_round['fight_id'] = stats_by_round['rd_id'].str.split('-').str[0]
stats_by_round= stats_by_round.merge(fight_details[['fight_id', 'rds_sched']], on='fight_id')

# Group round stats by fight for later use
grouped_by_fight = stats_by_round.groupby(['fighter_id', 'fight_id']).sum().reset_index()

# Merge recency information to stats_by_round table and grouped_by_fight table
stats_by_round['fight_fighter'] = stats_by_round['fight_id'] + '-' + stats_by_round['fighter_id']
stats_by_round = stats_by_round.merge(fight_recency[['fight_fighter', 'event_date', 'recency']],
                                      on='fight_fighter')

grouped_by_fight['fight_fighter'] = grouped_by_fight['fight_id'] + '-' + grouped_by_fight['fighter_id']
grouped_by_fight = grouped_by_fight.merge(fight_recency[['fight_fighter', 'event_date', 'recency']],
                                          on='fight_fighter')

## Group Stats by Fighter and Round of the Fight

In [10]:
def group_stats(stats_by_round, recency_max=1000):
    """
    Group stats by round and fighter, filtered by provided recency.
    """
    stats_by_round = stats_by_round[stats_by_round['recency'] <= recency_max]
    grouped_by_round = stats_by_round.groupby(['fighter_id', 'rd_num']).sum().reset_index()
    grouped_by_fighter = stats_by_round.groupby('fighter_id').sum().reset_index()
    
    return grouped_by_round, grouped_by_fighter

In [11]:
grouped_by_round_all, grouped_by_fighter_all = group_stats(stats_by_round.copy())
grouped_by_round_past_10, grouped_by_fighter_past_10 = group_stats(stats_by_round.copy(), 10)
grouped_by_round_past_5, grouped_by_fighter_past_5 = group_stats(stats_by_round.copy(), 5)

## Calculate Clustering Metrics

#### Per Minute of Control/Neutral Time

In [12]:
def per_min_ctrl_neutral(grouped_by_fighter, by_fight=False): 
    # Calculate stats per minute of control/neutral time
    per_ctrl_plus_neutral_cols = ['fighter_id', 'kd', 'kd_taken', 'total_sig_land', 'total_sig_att', 'total_strike_land', 'total_strike_att',
                                  'total_sig_land_opp', 'total_strike_taken', 'sub_att', 'ctrl_plus_neutral']
    
    # New column names
    new_cols = ['fighter_id', 'kd_per_min', 'kd_taken_per_min', 'sig_land_per_min', 'sig_att_per_min', 'strike_land_per_min',
                'strike_att_per_min', 'sig_absorbed_per_min', 'strike_absorbed_per_min', 'sub_att_per_min', 'ctrl_plus_neutral'
               ]
    
    offset = 1
    
    if by_fight:
        per_ctrl_plus_neutral_cols.insert(1, 'fight_id')
        new_cols.insert(1, 'fight_id')
        offset = 2

    per_ctrl_plus_neutral = grouped_by_fighter[per_ctrl_plus_neutral_cols].copy(deep=True)

    per_ctrl_plus_neutral.iloc[:, offset:] = per_ctrl_plus_neutral.iloc[:, offset:].div(per_ctrl_plus_neutral['ctrl_plus_neutral'], axis=0).multiply(60, axis=0)

    # Rename columns to reflect "per minute"
    per_ctrl_plus_neutral.columns = new_cols
    
    return per_ctrl_plus_neutral

In [13]:
per_ctrl_plus_neutral_all = per_min_ctrl_neutral(grouped_by_fighter_all)
per_ctrl_plus_neutral_past_10 = per_min_ctrl_neutral(grouped_by_fighter_past_10)
per_ctrl_plus_neutral_past_5 = per_min_ctrl_neutral(grouped_by_fighter_past_5)

per_ctrl_plus_neutral_by_fight = per_min_ctrl_neutral(grouped_by_fight, True)

#### Per Minute of Neutral Time

In [14]:
def per_min_neutral(grouped_by_fighter, by_fight=False):
    # Calculate stats per minute of neutral time
    per_neutral_cols = ['fighter_id', 'takedown_att', 'takedown_land', 'takedown_taken', 'neutral_time']

    # Renaming columns
    new_cols = ['fighter_id', 'takedown_att_per_min', 'takedown_land_per_min', 'takedown_absorbed_per_min', 'neutral_time']
    
    offset = 1
    
    if by_fight:
        per_neutral_cols.insert(1, 'fight_id')
        new_cols.insert(1, 'fight_id')
        offset = 2
    
    per_neutral = grouped_by_fighter[per_neutral_cols].copy(deep=True)

    per_neutral.iloc[:, offset:] = per_neutral.iloc[:, offset:].div(per_neutral['neutral_time'], axis=0).multiply(60, axis=0)

    # Rename columns to reflect "per minute"
    per_neutral.columns = new_cols
    
    return per_neutral

In [15]:
per_neutral_all = per_min_neutral(grouped_by_fighter_all)
per_neutral_past_10 = per_min_neutral(grouped_by_fighter_past_10)
per_neutral_past_5 = per_min_neutral(grouped_by_fighter_past_5)

per_neutral_by_fight = per_min_neutral(grouped_by_fight, True)

#### Per Minute of Total Time

In [16]:
def per_min_total(grouped_by_fighter, by_fight=False):
    # Calculate stats per minute of total time (rd_length)
    per_total_time_cols = ['fighter_id', 'ctrl_time', 'ctrl_time_taken', 'rd_length']

    # Renaming columns
    new_cols = ['fighter_id', 'ctrl_time_per_min', 'ctrl_time_opp_per_min', 'rd_length']
    
    offset = 1
    
    if by_fight:
        per_total_time_cols.insert(1, 'fight_id')
        new_cols.insert(1, 'fight_id')
        offset = 2
    
    per_total_time = grouped_by_fighter[per_total_time_cols].copy(deep=True)

    per_total_time.iloc[:, offset:] = per_total_time.iloc[:, offset:].div(per_total_time['rd_length'], axis=0).multiply(60, axis=0)
    
    # Rename columns to reflect "per minute"
    per_total_time.columns = new_cols
    
    return per_total_time

In [17]:
per_total_time_all = per_min_total(grouped_by_fighter_all)
per_total_time_past_10 = per_min_total(grouped_by_fighter_past_10)
per_total_time_past_5 = per_min_total(grouped_by_fighter_past_5)

per_total_time_by_fight = per_min_total(grouped_by_fight, True)

#### Fight Metric Ratios

In [18]:
def metric_ratio_calcs(grouped_by_fighter, by_fight=False):
    # Copy the stats by fighter table
    metric_ratios = grouped_by_fighter.copy(deep=True)

    # Create fight metric ratio columns
    metric_ratios['ctrl_time_ratio'] = metric_ratios['ctrl_time'] / metric_ratios['ctrl_time_taken']
    metric_ratios['takedown_perc'] = metric_ratios['takedown_land'] / metric_ratios['takedown_att']
    metric_ratios['opp_takedown_perc'] = metric_ratios['takedown_taken'] / metric_ratios['takedown_seen']
    metric_ratios['leg_strike_rate'] = metric_ratios['sig_leg_att'] / metric_ratios['total_sig_att']
    metric_ratios['body_strike_rate'] = metric_ratios['sig_body_att'] / metric_ratios['total_sig_att']
    metric_ratios['head_strike_rate'] = metric_ratios['sig_head_att'] / metric_ratios['total_sig_att']
    metric_ratios['dist_strike_rate'] = metric_ratios['sig_dist_att'] / metric_ratios['total_sig_att']
    metric_ratios['clinch_strike_rate'] = metric_ratios['sig_clinch_att'] / metric_ratios['total_sig_att']
    metric_ratios['ground_strike_rate'] = metric_ratios['sig_ground_att'] / metric_ratios['total_sig_att']
    metric_ratios['leg_strike_acc'] = metric_ratios['sig_leg_land'] / metric_ratios['sig_leg_att']
    metric_ratios['body_strike_acc'] = metric_ratios['sig_body_land'] / metric_ratios['sig_body_att']
    metric_ratios['head_strike_acc'] = metric_ratios['sig_head_land'] / metric_ratios['sig_head_att']
    metric_ratios['dist_strike_acc'] = metric_ratios['sig_dist_land'] / metric_ratios['sig_dist_att']
    metric_ratios['clinch_strike_acc'] = metric_ratios['sig_clinch_land'] / metric_ratios['sig_clinch_att']
    metric_ratios['ground_strike_acc'] = metric_ratios['sig_ground_land'] / metric_ratios['sig_ground_att']
    metric_ratios['sig_strike_acc'] = metric_ratios['total_sig_land'] / metric_ratios['total_sig_att']
    metric_ratios['opp_sig_strike_acc'] = metric_ratios['total_sig_land_opp'] / metric_ratios['total_sig_att_opp']
    metric_ratios['total_strike_acc'] = metric_ratios['total_strike_land'] / metric_ratios['total_strike_att']
    metric_ratios['opp_total_strike_acc'] = metric_ratios['total_strike_taken'] / metric_ratios['total_strike_seen']
    metric_ratios['kd_per_sig_strike'] = metric_ratios['kd'] / metric_ratios['total_sig_att']
    metric_ratios['kd_per_sig_head_strike'] = metric_ratios['kd'] / metric_ratios['sig_head_att']
    metric_ratios['opp_kd_per_sig_strike'] = metric_ratios['kd_taken'] / metric_ratios['total_sig_att_opp']
    metric_ratios['opp_kd_per_sig_head_strike'] = metric_ratios['kd_taken'] / metric_ratios['sig_head_seen']
    
    if by_fight:
        metric_ratios = metric_ratios[metric_ratio_cols_by_fight]
    else:
        metric_ratios = metric_ratios[metric_ratio_cols]
    
    return metric_ratios

In [19]:
# Filter the metric ratios table to ratio columns only
metric_ratio_cols = ['fighter_id',
                     'ctrl_time_ratio',
                     'takedown_perc',
                     'opp_takedown_perc',
                     'leg_strike_rate',
                     'body_strike_rate',
                     'head_strike_rate',
                     'dist_strike_rate',
                     'clinch_strike_rate',
                     'ground_strike_rate',
                     'leg_strike_acc',
                     'body_strike_acc',
                     'head_strike_acc',
                     'dist_strike_acc',
                     'clinch_strike_acc',
                     'ground_strike_acc',
                     'sig_strike_acc',
                     'opp_sig_strike_acc',
                     'total_strike_acc',
                     'opp_total_strike_acc',
                     'kd_per_sig_strike',
                     'kd_per_sig_head_strike',
                     'opp_kd_per_sig_strike',
                     'opp_kd_per_sig_head_strike']

metric_ratio_cols_by_fight = metric_ratio_cols.copy()
metric_ratio_cols_by_fight.insert(1, 'fight_id')

In [20]:
metric_ratios_all = metric_ratio_calcs(grouped_by_fighter_all)
metric_ratios_past_10 = metric_ratio_calcs(grouped_by_fighter_past_10)
metric_ratios_past_5 = metric_ratio_calcs(grouped_by_fighter_past_5)

metric_ratios_by_fight = metric_ratio_calcs(grouped_by_fight, True)

#### Finish Metrics

In [21]:
def finish_metrics(finish_details, grouped_by_fighter, recency_max=1000):
    # Group finish details by method and count instances of winner and method combinations
    finish_details = finish_details[finish_details['recency'] <= recency_max].copy()
    finish_counts = finish_details.groupby(['fighter_id', 'result_method']).size().unstack(fill_value=0).reset_index()

    # Merge sub attempts columns
    sub_cols = ['fighter_id', 'sub_att', 'sub_seen']
    finish_counts = finish_counts.merge(grouped_by_fighter[sub_cols]) 

    # Create aggregate columns for win/loss rate calculations
    finish_counts['finish_wins'] = finish_counts['win - tko'] + finish_counts['win - sub']
    finish_counts['finish_losses'] = finish_counts['loss - tko'] + finish_counts['loss - sub']
    finish_counts['num_fights'] = finish_counts.iloc[:, 1:8].sum(axis=1)
    finish_counts['num_wins'] = finish_counts['win - decision'] + finish_counts['win - sub'] + finish_counts['win - tko']
    finish_counts['num_losses'] = finish_counts['loss - decision'] + finish_counts['loss - sub'] + finish_counts['loss - tko']

    # Calculate LOSS rates by different methods, both in terms of versus total fights and total wins
    finish_counts['tko_win_per_fight'] = finish_counts['win - tko'] / finish_counts['num_fights']
    finish_counts['sub_win_per_fight'] = finish_counts['win - sub'] / finish_counts['num_fights']
    finish_counts['dec_win_per_fight'] = finish_counts['win - decision'] / finish_counts['num_fights']
    finish_counts['tko_win_per_win'] = finish_counts['win - tko'] / finish_counts['num_wins']
    finish_counts['sub_win_per_win'] = finish_counts['win - sub'] / finish_counts['num_wins']
    finish_counts['dec_win_per_win'] = finish_counts['win - decision'] / finish_counts['num_wins']

    # Calculate LOSS rates by different methods, both in terms of versus total fights and total wins
    finish_counts['tko_loss_per_fight'] = finish_counts['loss - tko'] / finish_counts['num_fights']
    finish_counts['sub_loss_per_fight'] = finish_counts['loss - sub'] / finish_counts['num_fights']
    finish_counts['dec_loss_per_fight'] = finish_counts['loss - decision'] / finish_counts['num_fights']
    finish_counts['tko_loss_per_loss'] = finish_counts['loss - tko'] / finish_counts['num_losses']
    finish_counts['sub_loss_per_loss'] = finish_counts['loss - sub'] / finish_counts['num_losses']
    finish_counts['dec_loss_per_loss'] = finish_counts['loss - decision'] / finish_counts['num_losses']

    # Calculate submission success rate for fighter and opponent  (DOESN'T CALC PROPERLY)
    #finish_counts['sub_success_rate'] = finish_counts['win - sub'] / finish_counts['sub_att']
    #finish_counts['opp_sub_success_rate'] = finish_counts['loss - sub'] / finish_counts['sub_seen']

    # Calculate total finish rate and opponent finish rate
    finish_counts['finish_rate'] = finish_counts['finish_wins'] / finish_counts['num_fights']
    finish_counts['opp_finish_rate'] = finish_counts['finish_losses'] / finish_counts['num_fights']

    # Filter to final columns
    final_finish_cols = ['fighter_id', 'num_fights', 'tko_win_per_fight', 'sub_win_per_fight', 'dec_win_per_fight',
                         'tko_win_per_win', 'sub_win_per_win', 'dec_win_per_win', 'tko_loss_per_fight',
                         'sub_loss_per_fight', 'dec_loss_per_fight', 'tko_loss_per_loss', 'sub_loss_per_loss',
                         'dec_loss_per_loss', 'finish_rate',
                         'opp_finish_rate'
                        ]

    finish_counts = finish_counts[final_finish_cols]
    
    return finish_counts

In [22]:
# Adjust fight detail table such that each row represents a fighter's performance in a single fight
finish_details_f1 = fight_details[['fight_id', 'method', 'fighter_1_id', 'winner_id']].copy()
finish_details_f2 = fight_details[['fight_id', 'method', 'fighter_2_id', 'winner_id']].copy()

finish_details_f1.columns = ['fight_id', 'method', 'fighter_id', 'winner_id']
finish_details_f2.columns = ['fight_id', 'method', 'fighter_id', 'winner_id']

finish_details = pd.concat([finish_details_f1, finish_details_f2]).sort_values('fight_id').reset_index(drop=True)

In [23]:
# Replace values in the method column
method_dict = {'Decision - Unanimous' : 'decision',
               'Decision - Split' : 'decision',
               'Decision - Majority' : 'decision',
               'KO/TKO' : 'tko',
               'TKO - Doctor\'s Stoppage' : 'tko',
               'Could Not Continue	' : 'exclude',
               'Could Not Continue' : 'exclude',
               'Overturned' : 'exclude',
               'Other' : 'exclude',
               'DQ' : 'exclude',
               'Submission' : 'sub'
              }

finish_details.replace({'method': method_dict}, inplace=True)

In [24]:
# Add results column to reflect win/loss/draw and method
finish_details['result'] = np.where(finish_details['fighter_id']==finish_details['winner_id'], 'win',
                                    np.where(finish_details['winner_id']=='Draw', 'draw',
                                             np.where(finish_details['winner_id']=='NC', 'no contest', 'loss')                                                    
                                            )
                                   )

finish_details_by_fight = finish_details.copy(deep=True)

finish_details = finish_details[(~finish_details['result'].isin(['no contest'])) &
                                (~finish_details['method'].isin(['exclude']))
                               ]

finish_details['result_method'] = finish_details['result'] + ' - ' + finish_details['method']
finish_details_by_fight['result_method']=np.where(finish_details_by_fight['result']=='no contest', 'no contest',
                                                  finish_details_by_fight['result'] + ' - ' + finish_details_by_fight['method']
                                                 )

In [25]:
# Merge recency information to finish_details table
finish_details['fight_fighter'] = finish_details['fight_id'] + '-' + finish_details['fighter_id']
finish_details = finish_details.merge(fight_recency[['fight_fighter', 'event_date', 'recency']],
                                      on='fight_fighter')

In [26]:
finish_counts_all = finish_metrics(finish_details.copy(), grouped_by_fighter_all)
finish_counts_past_10 = finish_metrics(finish_details.copy(), grouped_by_fighter_past_10, 10)
finish_counts_past_5 = finish_metrics(finish_details.copy(), grouped_by_fighter_past_5, 5)

In [27]:
# Create finish table by fight
finish_by_fight = finish_details_by_fight.pivot_table(index=['fight_id', 'fighter_id'], columns='result_method', values='result', aggfunc='count').reset_index()
finish_by_fight.fillna(0, inplace=True)

#### Control and Strike Pacing

In [28]:
# Tag rounds as early or late
stats_by_round['early_late'] = np.where(stats_by_round['rds_sched']=='3', np.where(stats_by_round['rd_num']<=2, 'early', 'late'),
                                        np.where(stats_by_round['rd_num']<=3, 'early', 'late')
                                       )

In [29]:
# Establish columns for pace calculations
pace_cols = ['fighter_id', 'early_late', 'rd_length', 'ctrl_time', 'ctrl_time_taken', 'total_sig_land', 'total_sig_land_opp']

In [30]:
def pace_calcs(stats_by_round, recency_max=1000):
    
    stats_by_round = stats_by_round[stats_by_round['recency'] <= recency_max]
    pace_metrics = stats_by_round[pace_cols].groupby(['fighter_id', 'early_late']).sum().reset_index()
    
    # Split into two tables, early and late rounds
    pace_metrics = pace_metrics.sort_values(['early_late', 'fighter_id'])
    early = pace_metrics[pace_metrics['early_late']=='early'].reset_index(drop=True)
    late = pace_metrics[pace_metrics['early_late']=='late'].reset_index(drop=True)

    # Rename columns
    early.columns = ['fighter_id', 'early_late', 'rd_length_early', 'ctrl_time_early', 'ctrl_time_taken_early', 'total_sig_land_early', 'total_sig_land_opp_early']
    late.columns = ['fighter_id', 'early_late', 'rd_length_late', 'ctrl_time_late', 'ctrl_time_taken_late', 'total_sig_land_late', 'total_sig_land_opp_late']

    # Recombine early and late metrics
    pace_metrics = late.merge(early, on='fighter_id')

    # Calculate pacing metrics for control and strike rates in early vs late rounds
    pace_metrics['control_rate_early'] = pace_metrics['ctrl_time_early'] / pace_metrics['rd_length_early']
    pace_metrics['control_rate_late'] = pace_metrics['ctrl_time_late'] / pace_metrics['rd_length_late']
    pace_metrics['control_rate_late_vs_early'] = pace_metrics['control_rate_late'] / pace_metrics['control_rate_early']
    pace_metrics['sig_strike_rate_early'] = pace_metrics['total_sig_land_early'] / pace_metrics['rd_length_early']
    pace_metrics['sig_strike_rate_late'] = pace_metrics['total_sig_land_late'] / pace_metrics['rd_length_late']
    pace_metrics['sig_strike_rate_late_vs_early'] = pace_metrics['sig_strike_rate_late'] / pace_metrics['sig_strike_rate_early']

    # Filter to the final table of pace metrics
    final_pace_cols = ['fighter_id', 'control_rate_late_vs_early', 'sig_strike_rate_late_vs_early']
    pace_metrics = pace_metrics[final_pace_cols]
    
    return pace_metrics

In [31]:
pace_metrics_all = pace_calcs(stats_by_round.copy())
pace_metrics_past_10 = pace_calcs(stats_by_round.copy(), 10)
pace_metrics_past_5 = pace_calcs(stats_by_round.copy(), 5)

## Merge Clustering Metrics

In [32]:
def merge_metrics(finish_counts, per_ctrl_plus_neutral, per_neutral, per_total_time, metric_ratios, pace_metrics=None, by_fight=False):
    """
    Merge all fighter metrics.
    """
    
    if by_fight:
        merge_col = ['fighter_id',  'fight_id']
    else:
        merge_col = 'fighter_id'
    
    merged_metrics = finish_counts.copy()
    merged_metrics = merged_metrics.merge(per_ctrl_plus_neutral.iloc[:, 0:-1], on=merge_col, how='left')
    merged_metrics = merged_metrics.merge(per_neutral.iloc[:, 0:-1], on=merge_col, how='left')
    merged_metrics = merged_metrics.merge(per_total_time.iloc[:, 0:-1], on=merge_col, how='left')
    merged_metrics = merged_metrics.merge(metric_ratios, on=merge_col, how='left')
    
    if by_fight==False:
        merged_metrics = merged_metrics.merge(pace_metrics, on=merge_col, how='left')
    
    return merged_metrics

In [33]:
merged_metrics_all = merge_metrics(finish_counts_all, per_ctrl_plus_neutral_all, per_neutral_all, 
                                   per_total_time_all, metric_ratios_all, pace_metrics_all)

merged_metrics_past_10 = merge_metrics(finish_counts_past_10, per_ctrl_plus_neutral_past_10, per_neutral_past_10, 
                                       per_total_time_past_10, metric_ratios_past_10, pace_metrics_past_10)

merged_metrics_past_5 = merge_metrics(finish_counts_past_5, per_ctrl_plus_neutral_past_5, per_neutral_past_5, 
                                      per_total_time_past_5, metric_ratios_past_5, pace_metrics_past_5)

merged_metrics_by_fight = merge_metrics(finish_by_fight, per_ctrl_plus_neutral_by_fight, per_neutral_by_fight, 
                                        per_total_time_by_fight, metric_ratios_by_fight, by_fight=True)

## Clean Merged Metrics

In [34]:
def clean_up_metrics(merged_metrics):
    """
    Replace nulls and infinities.
    """
    
    # Replace NaN with zero
    merged_metrics = merged_metrics.fillna(0)
    
    # Replace infinity with zero
    merged_metrics = merged_metrics.replace([np.inf, -np.inf], 0)
    
    return merged_metrics

In [35]:
# Clean up all fight metrics (nan, infinity)
merged_metrics_all = clean_up_metrics(merged_metrics_all)
merged_metrics_past_10 = clean_up_metrics(merged_metrics_past_10)
merged_metrics_past_5 = clean_up_metrics(merged_metrics_past_5)
merged_metrics_by_fight = clean_up_metrics(merged_metrics_by_fight)

## Cluster Assignments

### *Import Scaler, PCA, and K-Means Model*

In [36]:
# Import K-Means model
with open('nf_km_10.pickle', 'rb') as read_file:
    km = pickle.load(read_file)
    
# Import PCA
with open('nf_pca_fighter_style.pickle', 'rb') as read_file:
    pca = pickle.load(read_file)
    
# Import Standard Scaler
with open('nf_standard_scaler_fighter_style.pickle', 'rb') as read_file:
    st_scale = pickle.load(read_file)

### *Filter and Transform Data for Clustering*

In [37]:
def scale_filter_and_transform(fight_metrics):
    """
    Filter for clustering columns and transform using PCA used in initial cluster formation.
    """
    
    fight_metrics = fight_metrics[cluster_cols].copy()
    fight_metrics_st = st_scale.transform(fight_metrics)
    fight_metrics_pca = pca.transform(fight_metrics_st)
    
    return fight_metrics_pca

In [38]:
cluster_cols = ['head_strike_rate', 'dist_strike_rate', 'sig_att_per_min',
                'kd_per_sig_strike', 'sig_strike_acc', 'total_strike_acc',
                'opp_kd_per_sig_strike', 'opp_sig_strike_acc', 'sig_absorbed_per_min',
                'takedown_land_per_min', 'ctrl_time_per_min', 'sub_att_per_min',
                'takedown_absorbed_per_min', 'opp_takedown_perc'
               ]

In [39]:
pca_all = scale_filter_and_transform(merged_metrics_all)
pca_past_5 = scale_filter_and_transform(merged_metrics_past_5)
pca_past_10 = scale_filter_and_transform(merged_metrics_past_10)

## Cluster Breakdown by Fighter

In [40]:
def cluster_breakdown(cluster_data_pca):
    """
    For given PCA data, return a cluster percentage breakdown for each fighter.
    """
    # Calculate Euclidean distance from fighters to each cluster
    nodes_to_centroids = distance.cdist(cluster_data_pca, centroids, 'euclidean')
    
    # Set up dataframe of distances to cluster centroids
    nodes_to_centroids_df = pd.DataFrame(nodes_to_centroids, columns=breakdown_cols)

    # Convert distances to "similiarity scores"
    nodes_to_centroids_df['max_distance'] = nodes_to_centroids_df.max(axis=1)
    nodes_to_centroids_df = nodes_to_centroids_df.div(nodes_to_centroids_df['max_distance'], axis=0)
    nodes_to_centroids_df = 1 - nodes_to_centroids_df

    # Convert similarity scores to percentages for each cluster
    nodes_to_centroids_df['sum'] = nodes_to_centroids_df.sum(axis=1)
    nodes_to_centroids_df = nodes_to_centroids_df.div(nodes_to_centroids_df['sum'], axis=0)

    # Eliminate unnecessary columns, calculate dominant cluster
    cluster_breakdown = nodes_to_centroids_df[breakdown_cols]
    cluster_breakdown['max_percentage'] = cluster_breakdown.max(axis=1)
    cluster_breakdown['top_cluster'] = km.predict(cluster_data_pca)
    
    return cluster_breakdown

In [41]:
# Pull centroid positions from K-Means model
centroids = km.cluster_centers_

# Establish columns for cluster breakdown dataframe
breakdown_cols = ['Cluster 0',
                  'Cluster 1',
                  'Cluster 2',
                  'Cluster 3',
                  'Cluster 4',
                  'Cluster 5',
                  'Cluster 6',
                  'Cluster 7',
                  'Cluster 8',
                  'Cluster 9',
                 ]

In [42]:
# Calculate cluster breakdowns
cluster_breakdown_all = cluster_breakdown(pca_all)
cluster_breakdown_past_10 = cluster_breakdown(pca_past_10)
cluster_breakdown_past_5 = cluster_breakdown(pca_past_5)

## Prepare Data for Tableau

In [43]:
def stack_with_tag(data_list, tag_list):
    
    df = data_list[0].copy()
    df['past_fights'] = tag_list[0]
    
    for i in range(1, len(data_list)):
        df_2 = data_list[i].copy()
        df_2['past_fights'] = tag_list[i]
        df = pd.concat([df, df_2]).reset_index(drop=True)
    
    return df

### *Fight Metrics*

In [44]:
# Stack and tag fight metrics
fighter_metrics_tableau = stack_with_tag([merged_metrics_all, merged_metrics_past_10, merged_metrics_past_5],
                                         ['All', 'Past 10', 'Past 5'])

### *Cluster Data*

In [45]:
# Stack and tag cluster breakdowns
cluster_breakdown_all['fighter_id'] = merged_metrics_all['fighter_id']
cluster_breakdown_past_10['fighter_id'] = merged_metrics_past_10['fighter_id']
cluster_breakdown_past_5['fighter_id'] = merged_metrics_past_5['fighter_id']

cluster_breakdown_stacked = stack_with_tag([cluster_breakdown_all, cluster_breakdown_past_10, cluster_breakdown_past_5],
                                           ['All', 'Past 10', 'Past 5'])

# Melt table
cluster_breakdown_tableau = cluster_breakdown_stacked[['fighter_id', 'past_fights']  + breakdown_cols].melt(id_vars=['fighter_id', 'past_fights'], 
                                                                                                            var_name='cluster',
                                                                                                            value_name='percentage')
# Save copy for later
cluster_breakdown = cluster_breakdown_tableau.copy()

# Map cluster values to cluster names:
cluster_map = {'Cluster 0' : 'High Risk Sub Artist',
               'Cluster 1' : 'Patient Power Puncher',
               'Cluster 2' : 'Stick and Move',
               'Cluster 3' : 'Pressure Wrestler',
               'Cluster 4' : 'Head Hunting Wrestler',
               'Cluster 5' : 'Glass Cannon',
               'Cluster 6' : 'Stand and Bang',
               'Cluster 7' : 'Grind It Out',
               'Cluster 8' : 'Chinny Grappler',
               'Cluster 9' : 'Tactician'
              }

cluster_breakdown_tableau['f1_cluster'] = cluster_breakdown_tableau['cluster'].map(cluster_map)
cluster_breakdown_tableau['cluster'] = cluster_breakdown_tableau['cluster'].map(cluster_map)
cluster_breakdown['cluster'] = cluster_breakdown['cluster'].map(cluster_map)

In [46]:
# Grab top clusters for each fighter 
top_clusters = cluster_breakdown_all[['fighter_id', 'top_cluster']]
top_clusters_past_10 = cluster_breakdown_past_10[['fighter_id', 'top_cluster']]
top_clusters_past_5 = cluster_breakdown_past_5[['fighter_id', 'top_cluster']]

# Map top clusters to cluster names
cluster_map_int = {0 : 'High Risk Sub Artist',
                   1 : 'Patient Power Puncher',
                   2 : 'Stick and Move',
                   3 : 'Pressure Wrestler',
                   4 : 'Head Hunting Wrestler',
                   5 : 'Glass Cannon',
                   6 : 'Stand and Bang',
                   7 : 'Grind It Out', 
                   8 : 'Chinny Grappler',
                   9 : 'Tactician',
                  }

top_clusters['top_cluster'] = top_clusters['top_cluster'].map(cluster_map_int)
top_clusters_past_10['top_cluster'] = top_clusters_past_10['top_cluster'].map(cluster_map_int)
top_clusters_past_5['top_cluster'] = top_clusters_past_5['top_cluster'].map(cluster_map_int)

In [47]:
# Start with fighter ID's
style_matchups = fight_details[['fighter_1_id', 'fighter_2_id', 'winner_id']]

In [48]:
# Add loser ID
style_matchups['loser_id'] = np.where(style_matchups['winner_id']==style_matchups['fighter_1_id'],
                                      style_matchups['fighter_2_id'], style_matchups['fighter_1_id'])

# Merge number of fights and top clusters for winner and loser ID's
style_matchups = style_matchups.merge(merged_metrics_all[['fighter_id', 'num_fights']], left_on='winner_id', right_on='fighter_id')
style_matchups = style_matchups.merge(merged_metrics_all[['fighter_id', 'num_fights']], left_on='loser_id', right_on='fighter_id', suffixes=['_winner', '_loser'])

style_matchups = style_matchups.merge(top_clusters, left_on='winner_id', right_on='fighter_id')
style_matchups = style_matchups.merge(top_clusters, left_on='loser_id', right_on='fighter_id', suffixes=['_winner', '_loser'])

In [49]:
style_matchups = style_matchups[(style_matchups['num_fights_winner']>=10) & (style_matchups['num_fights_loser']>=10)][['top_cluster_winner', 'top_cluster_loser']].reset_index(drop=True)

In [50]:
# Group by winner and loser ID's to determine which clusters win over others in their matchups
winners = style_matchups.groupby(['top_cluster_winner', 'top_cluster_loser']).size().unstack(fill_value=0)
losers = style_matchups.groupby(['top_cluster_loser', 'top_cluster_winner']).size().unstack(fill_value=0)

# Calculate win percentage by cluster matchup
win_percentage = winners / (winners + losers)

In [51]:
# Tidy up win percentage, rename columns, and map to cluster labels
win_percentage = win_percentage.reset_index().melt(id_vars='top_cluster_winner')
win_percentage.columns = ['f1_cluster', 'f2_cluster', 'win_percentage']

#win_percentage['f1_cluster'] = win_percentage['f1_cluster'].map(cluster_map)
#win_percentage['f2_cluster'] = win_percentage['f2_cluster'].map(cluster_map)

In [52]:
cluster_breakdown_tableau = cluster_breakdown_tableau.merge(win_percentage, on='f1_cluster')

In [53]:
cluster_breakdown_tableau = cluster_breakdown_tableau.merge(cluster_breakdown[['fighter_id', 'past_fights', 'cluster', 'percentage']],
                                                            left_on=['fighter_id', 'past_fights', 'f2_cluster'], 
                                                            right_on=['fighter_id', 'past_fights', 'cluster'],
                                                            suffixes=['','_f2'])

In [54]:
cluster_breakdown_tableau.drop(columns=['cluster_f2'], inplace=True)

### *Fighter Details*

In [55]:
# Set up fighter detail table
fighter_cols = ['fighter_id', 'first_name', 'last_name', 'nickname', 'gender', 'reach', 'height', 
                'stance', 'wins', 'losses', 'draws', 'belt', 'last_fight_date', 'latest_weight_class']

fighter_details_tableau = fighter_details[fighter_cols]
fighter_details_tableau['full_name'] = np.where(fighter_details_tableau['first_name'].isnull(),
                                                fighter_details_tableau['last_name'],
                                                fighter_details_tableau['first_name'] + ' ' + fighter_details_tableau['last_name']
                                               )

# Included flipped name (last first) for merging with fighter rankings later
fighter_details_tableau['full_name_flipped'] = np.where(fighter_details_tableau['first_name'].isnull(),
                                                        fighter_details_tableau['last_name'],
                                                        fighter_details_tableau['last_name'] + ' ' + fighter_details_tableau['first_name']
                                               )

In [56]:
# Merge top cluster detail (all, past 5, past 10) and set column names
fighter_details_tableau = fighter_details_tableau.merge(top_clusters, on='fighter_id')
fighter_details_tableau = fighter_details_tableau.merge(top_clusters_past_10, on='fighter_id', suffixes=('_all', '_past_10'))
fighter_details_tableau = fighter_details_tableau.merge(top_clusters_past_10, on='fighter_id')
fighter_details_tableau = fighter_details_tableau.rename(columns={'top_cluster' : 'top_cluster_past_5'})

### *Fight Details*

In [57]:
# Merge fight details to the fight recency table
fight_detail_cols = ['fight_id', 'fighter_1_id', 'fighter_2_id', 'winner_id', 'weightclass', 'rds_sched', 'rd_ended', 'method', 'bonus']

fight_details_tableau = fight_recency.merge(fight_details[fight_detail_cols], on='fight_id')

In [58]:
# Determine opponent fighter_id
fight_details_tableau['opponent_id'] = np.where(fight_details_tableau['fighter_id']==fight_details_tableau['fighter_1_id'], 
                                                fight_details_tableau['fighter_2_id'], fight_details_tableau['fighter_1_id'])

# Add opponent full name to the fight details table
fight_details_tableau = fight_details_tableau.merge(fighter_details_tableau[['fighter_id', 'full_name', 'full_name_flipped']],
                                                    left_on='opponent_id', right_on='fighter_id', suffixes=('', '_y'))

fight_details_tableau.rename(columns={'full_name' : 'opponent_name',
                                      'full_name_flipped' : 'opponent_name_flipped'
                                     }, inplace=True)

In [59]:
# Import rankings snapshots
with open('rankings_snapshots.pickle', 'rb') as read_file:
    snapshots = pickle.load(read_file)

In [60]:
# Convert date columns to datetime in rankings snapshots and fight detail tables, sort ascending
fight_details_tableau['event_date'] = pd.to_datetime(fight_details_tableau['event_date'], format='%Y-%m-%d')
fight_details_tableau = fight_details_tableau.sort_values('event_date')

snapshots['date'] = pd.to_datetime(snapshots['date'], format='%m/%d/%Y')
snapshots = snapshots.sort_values(['date', 'rank'])

# Drop duplicates of event date and fighter name (keep highest ranking)
snapshot_no_dups = snapshots.drop_duplicates(subset=['date', 'fighter'])

In [61]:
# Replace fighter names to match rankings website
name_replace = {'Aleksei Oleinik' : 'Alexey Oleynik',
                'BJ Penn' : 'B.J. Penn',
                'Benson Henderson' : 'Ben Henderson',
                'C.B. Dollaway' : 'CB Dollaway',
                'Constantinos Philippou' : 'Costas Philippou',
                'Dooho Choi' : 'Doo Ho Choi',
                'Georges St-Pierre' : 'Georges St. Pierre',
                'JJ Aldrich' : 'J.J. Aldrich',
                'Jacare Souza' : 'Ronaldo Souza',
                'Jimy Hettes' : 'Jim Hettes',
                'Joe Duffy' : 'Joseph Duffy',
                'Jose Quinonez' : 'Jose Alberto Quinonez',
                'Kai Kara-France' : 'Kai Kara France',
                'Khalil Rountree Jr.' : 'Khalil Rountree',
                'Luis Henrique' : 'Luis Henrique Barbosa de Oliveira',
                'Manvel Gamburyan' : 'Manny Gamburyan',
                'Matthew Riddle' : 'Matt Riddle',
                'Mike Brown' : 'Mike Thomas Brown',
                'Ovince Saint Preux' : 'Ovince St. Preux',
                'Renato Moicano' : 'Renato Carneiro',
                'Rob Emerson' : 'Robert Emerson',
                'Robert Peralta' : 'Robbie Peralta',
                'Rogerio Nogueira' : 'Antonio Rogerio Nogueira',
                'Rony Jason' : 'Rony Mariano Bezerra',
                'Serghei Spivac' : 'Sergey Spivak',
                'SeungWoo Choi' : 'Seung Woo Choi',
                'TJ Dillashaw' : 'T.J. Dillashaw',
                'TJ Grant' : 'T.J. Grant',
                'TJ Waldburger' : 'T.J. Waldburger',
                'Khaos Williams' : 'Kalinn Williams'
               }

# Remap incorrect names using dictionary
fight_details_tableau['opponent_name_clean'] = fight_details_tableau['opponent_name']
fight_details_tableau['opponent_name_flipped_clean'] = fight_details_tableau['opponent_name_flipped']

fight_details_tableau['opponent_name_clean'] = fight_details_tableau['opponent_name_clean'].replace(name_replace)
fight_details_tableau['opponent_name_flipped_clean'] = fight_details_tableau['opponent_name_flipped_clean'].replace(name_replace)

In [62]:
# Create clean columns for merging (remove apostrophes, all lowercase, strip blanks)
def clean_column(df, col):
    """
    Replace apostrophes with blanks, convert to lowercase, and strip any blanks.
    """
    df[col] = df[col].replace({'\'': ''}, regex=True)
    df[col] = df[col].str.lower().str.strip()
    
clean_column(fight_details_tableau, 'opponent_name_clean')
clean_column(fight_details_tableau, 'opponent_name_flipped_clean')
clean_column(snapshot_no_dups, 'fighter')

In [63]:
# Merge rankings data based on the latest snapshot (merge "as of")
fight_details_tableau = pd.merge_asof(fight_details_tableau, snapshot_no_dups, left_on='event_date', right_on='date', left_by='opponent_name_clean',
                                      right_by='fighter')

fight_details_tableau = pd.merge_asof(fight_details_tableau, snapshot_no_dups, left_on='event_date', right_on='date', left_by='opponent_name_flipped_clean',
                                      right_by='fighter')

In [64]:
fight_details_tableau['opponent_rank'] = fight_details_tableau[['rank_x','rank_y']].min(axis=1)

In [65]:
# Determine result for each fighter and fight combination (win/loss/draw)
fight_details_tableau['result'] = np.where(fight_details_tableau['winner_id'].isin(['NC', 'Draw']), fight_details_tableau['winner_id'],
                                           np.where(fight_details_tableau['winner_id']==fight_details_tableau['fighter_id'], 'Win', 'Loss')
                                          )

In [66]:
# Create bonus detail column considering whether the fighter won or loss the fight
fight_details_tableau['bonus_detail'] = fight_details_tableau['bonus'].str.split(', ')

bonus_dict = {'winner' : {'belt' : 'Belt',
                          'ko' : 'KO',
                          'sub' : 'SUB',
                          'fight' : 'FOTN',
                          'perf' : 'POTN'
                         },
              'loser' : {'belt' : '',
                         'ko' : '',
                         'sub' : '',
                         'fight' : 'FOTN',
                         'perf' : ''
                        }
             }

fight_details_tableau['bonus_detail'] = fight_details_tableau.apply(lambda x: [] if x['bonus_detail']==None else 
                                                                    [bonus_dict['winner' if x['result']=='Win' else 'loser'][i] for i in x['bonus_detail']], axis=1)

fight_details_tableau['bonus_detail'] = fight_details_tableau['bonus_detail'].str.join(', ').str.strip().str.strip(',')
fight_details_tableau['bonus_detail'] = np.where(fight_details_tableau['bonus_detail']=='', None, fight_details_tableau['bonus_detail'])

In [67]:
# Filter tableau table to only necessary columns
tableau_fight_cols = ['fighter_id', 'fight_id', 'opponent_name', 'opponent_rank', 'weightclass', 'event_date', 'recency', 
                      'rds_sched', 'rd_ended', 'result', 'method', 'bonus_detail']
fight_details_tableau = fight_details_tableau[tableau_fight_cols]
fight_details_tableau = fight_details_tableau.merge(merged_metrics_by_fight, on=['fight_id', 'fighter_id'], how='left')

### *Historical Rankings*

In [68]:
# Clean snapshots and fighter_details columns for merging fighter_ids
clean_column(snapshots, 'fighter')

fighter_details_tableau['full_name_clean'] = fighter_details_tableau['full_name']
fighter_details_tableau['full_name_clean'] = fighter_details_tableau['full_name_clean'].replace(name_replace)
clean_column(fighter_details_tableau, 'full_name_clean')

clean_column(fighter_details_tableau, 'full_name_flipped')

In [69]:
# Merge fighter ids where possible
snapshots = snapshots.merge(fighter_details_tableau[['full_name_flipped', 'fighter_id']], left_on='fighter', right_on='full_name_flipped', how='left')
snapshots = snapshots.merge(fighter_details_tableau[['full_name_clean', 'fighter_id']], left_on='fighter', right_on='full_name_clean', how='left')

snapshots['fighter_id'] = np.where(snapshots['fighter_id_x'].isnull(), snapshots['fighter_id_y'], snapshots['fighter_id_x'])

In [70]:
# Filter columns for Tableau
rankings_tableau = snapshots[['date', 'division', 'rank', 'fighter_id']]

In [71]:
# Reset Tableau columns for fighter details table
fighter_details_tableau = fighter_details_tableau[fighter_cols + ['top_cluster_all', 'top_cluster_past_10', 'top_cluster_past_5'] + ['full_name']]

In [72]:
# Add a blank row for use in the fighter selection parameter in Tableau
fighter_details_tableau.loc[fighter_details_tableau.shape[0]] = [None] * len(fighter_details_tableau.columns)
fighter_details_tableau.iloc[-1, -1] = '(None Selected)'

### *Upcoming Event*

In [73]:
# Request upcoming events page
page = requests.get('http://ufcstats.com/statistics/events/upcoming').text
soup = BeautifulSoup(page, 'html5lib')

# Request html for the next event only
next_event = soup.find('table').find('a').get('href')

page = requests.get(next_event).text
soup = BeautifulSoup(page, 'html5lib')

# Get fighter details for next event
fighters = soup.find('table').find_all('a')

matchups = []

for i in range(0, len(fighters), 3):
    fighter_1 = fighters[i].text.strip()
    fighter_2 = fighters[i + 1].text.strip()
    
    matchups.append(fighter_1 + ' vs. ' + fighter_2)

# Save matchups for next event to a dataframe
tableau_matchups = pd.DataFrame({'Matchup' : matchups, 'Order' : range(len(matchups))})

### *Save Tables to Excel*

In [74]:
fighter_metrics_tableau.to_excel('tableau_fighter_metrics.xlsx', sheet_name='fighter_metrics')
cluster_breakdown_tableau.to_excel('tableau_cluster_breakdowns.xlsx', sheet_name='clusters')
fighter_details_tableau.to_excel('tableau_fighter_details.xlsx', sheet_name='fighter_details')
fight_details_tableau.to_excel('tableau_fight_details.xlsx', sheet_name='fight_details')
tableau_matchups.to_excel('tableau_matchups.xlsx', sheet_name='matchups', header=True)
rankings_tableau.to_excel('tableau_rankings.xlsx', sheet_name='rankings', header=True)